# Imports and parameters

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
import pandas as pd
from download import download_data_and_parse_it
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import train_test_split
import time


from sklearn.metrics import accuracy_score

batch_size = 32
num_classes = 3
learning_rate = 0.001
num_epochs = 10

# Create the model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define a simple CNN model
class SimpleCNN(nn.Module):
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


class QuickDrawDataset(Dataset):
    def __init__(self, drawings, labels):
        self.drawings = drawings
        self.labels = labels
        
    def __len__(self):
        return len(self.drawings)
    
    def __getitem__(self, idx):
        # Convert the drawing format to image
        drawing = self.drawings.iloc[idx] if isinstance(self.drawings, pd.Series) else self.drawings[idx]
        image = self.drawing_to_image(drawing)
        label = self.labels.iloc[idx] if isinstance(self.labels, pd.Series) else self.labels[idx]
        return torch.FloatTensor(image).unsqueeze(0), label
    
    def drawing_to_image(self, drawing):
        # Create a blank 64x64 image
        image = np.zeros((64, 64))
        
        # For each stroke in the drawing
        for stroke in drawing:
            # Get x and y coordinates
            x = stroke[0]
            y = stroke[1]
            
            # Draw lines between consecutive points
            for i in range(len(x)-1):
                x1, y1 = int(x[i]), int(y[i])
                x2, y2 = int(x[i+1]), int(y[i+1])
                
                # Ensure points are within bounds
                x1 = max(0, min(x1, 63))
                y1 = max(0, min(y1, 63))
                x2 = max(0, min(x2, 63))
                y2 = max(0, min(y2, 63))
                
                # Draw line
                image[y1, x1] = 255
                image[y2, x2] = 255
        
        return image / 255.0
    
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)
    

Using cuda device


# Data

In [30]:
le = LabelEncoder()
# Load the JSON file into a DataFrame
sword_dataset = download_data_and_parse_it("sword.ndjson")
tent_dataset = download_data_and_parse_it("tent.ndjson")
eiffel_dataset = download_data_and_parse_it("star.ndjson")
df = pd.concat([sword_dataset, tent_dataset])
df = pd.concat([df,eiffel_dataset])
df['class'] = le.fit_transform(df['word'])

# Split into training and testing sets
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# # Create datasets
# train_dataset = QuickDrawDataset(train_df['drawing'], train_df['class'])
# test_dataset = QuickDrawDataset(test_df['drawing'], test_df['class'])

# Split into training, validation, and testing sets
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)  # 60% train, 40% temp
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  # Split temp into 20% val, 20% test

# Create datasets
train_dataset = QuickDrawDataset(train_df['drawing'], train_df['class'])
val_dataset = QuickDrawDataset(val_df['drawing'], val_df['class'])
test_dataset = QuickDrawDataset(test_df['drawing'], test_df['class'])



# Create DataLoaders
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

The file exists.
         word countrycode                      timestamp  recognized  \
123792  sword          MY  2017-03-26 21:14:01.94413 UTC        True   
123793  sword          PL  2017-03-27 14:06:53.35221 UTC        True   
123794  sword          US  2017-03-17 18:29:46.34852 UTC        True   
123795  sword          US    2017-01-27 14:54:45.687 UTC        True   
123796  sword          SE   2017-01-27 09:38:24.6558 UTC        True   
123797  sword          PL  2017-01-30 20:47:58.83752 UTC        True   
123798  sword          DE  2017-01-27 13:56:28.54679 UTC        True   
123799  sword          CA  2017-03-28 13:42:02.27022 UTC        True   
123800  sword          DE  2017-03-12 09:35:38.31056 UTC        True   
123801  sword          AU  2017-03-03 06:18:46.69524 UTC        True   

                  key_id                                            drawing  
123792  5927313763991552  [[[13, 30, 66], [52, 0, 59]], [[11, 26, 53, 76...  
123793  6323617040171008  [[[38, 4

In [23]:
# Training loop
def train_model(model):
    model.train()
    for epoch in range(num_epochs):  # Number of epochs
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

# Testing loop
def test_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")


In [24]:
start_time = time.time()

train_model(model)
test_model(model)

end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.6f} seconds")

Epoch 1, Loss: 0.6875787117982629
Epoch 2, Loss: 0.6625695440808996
Epoch 3, Loss: 0.6541292833024788
Epoch 4, Loss: 0.6484258381668053
Epoch 5, Loss: 0.6437222478177144
Epoch 6, Loss: 0.6391067208073038
Epoch 7, Loss: 0.6351148034066738
Epoch 8, Loss: 0.6312979419840081
Epoch 9, Loss: 0.62712627709373
Epoch 10, Loss: 0.6237510833464539
Accuracy: 72.23%
Execution time: 418.718310 seconds


# Automation

## Define the models

In [35]:
# Define the dataset (dummy data for illustration)
class QuickDrawDataset(Dataset):
    def __init__(self, drawings, labels):
        self.drawings = drawings
        self.labels = labels

    def __len__(self):
        return len(self.drawings)

    def __getitem__(self, idx):
        drawing = self.drawings.iloc[idx]
        image = self.drawing_to_image(drawing)
        label = self.labels.iloc[idx]
        return torch.FloatTensor(image).unsqueeze(0), label

    def drawing_to_image(self, drawing):
        # Create a blank 64x64 image
        image = np.zeros((64, 64))
        
        # For each stroke in the drawing
        for stroke in drawing:
            # Get x and y coordinates
            x = stroke[0]
            y = stroke[1]
            
            # Draw lines between consecutive points
            for i in range(len(x)-1):
                x1, y1 = int(x[i]), int(y[i])
                x2, y2 = int(x[i+1]), int(y[i+1])
                
                # Ensure points are within bounds
                x1 = max(0, min(x1, 63))
                y1 = max(0, min(y1, 63))
                x2 = max(0, min(x2, 63))
                y2 = max(0, min(y2, 63))
                
                # Draw line
                image[y1, x1] = 255
                image[y2, x2] = 255

# Define a dynamic model class
def create_model(conv_layers, kernel_size, learning_rate, num_classes=10):
    class CustomCNN(nn.Module):
        def __init__(self):
            super(CustomCNN, self).__init__()
            layers = []
            in_channels = 1
            size = 64  # Initial image size
            for out_channels in conv_layers:
                layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1))
                layers.append(nn.ReLU())
                layers.append(nn.MaxPool2d(2))  # MaxPool2d halves the size
                in_channels = out_channels
                size = size // 2  # Each MaxPool2d halves the size
            
            self.conv_layers = nn.Sequential(*layers)

            # Dynamically calculate the output size after convolutions and pooling
            # Pass a dummy tensor through the convolution layers to get the output size
            with torch.no_grad():
                dummy_input = torch.zeros(1, 1, 64, 64)  # (batch_size, channels, height, width)
                output_size = self.conv_layers(dummy_input)
                # Output size is [batch_size, channels, height, width]
                final_size = output_size.view(1, -1).size(1)

            # Fully connected layers
            self.fc_layers = nn.Sequential(
                nn.Flatten(),
                nn.Linear(final_size, 128),
                nn.ReLU(),
                nn.Linear(128, num_classes)
            )

        def forward(self, x):
            x = self.conv_layers(x)
            x = self.fc_layers(x)
            return x

    return CustomCNN()

# Train and evaluate a model
def train_and_evaluate(config, train_loader, val_loader, device):
    model = create_model(config["conv_layers"], config["kernel_size"], config["learning_rate"], num_classes=10).to(device)
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    criterion = nn.CrossEntropyLoss()

    train_losses, val_accuracies = [], []
    for epoch in range(5):  # Keep epochs small for testing
        model.train()
        epoch_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        train_losses.append(epoch_loss / len(train_loader))

        # Validation
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_preds.extend(outputs.argmax(dim=1).tolist())
                val_labels.extend(labels.tolist())
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_accuracies.append(val_accuracy)

    return train_losses, val_accuracies

# Define the Optuna objective function
def objective(trial):
    conv_layers = trial.suggest_categorical("conv_layers", [[16, 32], [16, 32, 64], [32, 64]])
    kernel_size = trial.suggest_int("kernel_size", 3, 5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    config = {
        "conv_layers": conv_layers,
        "kernel_size": kernel_size,
        "learning_rate": learning_rate
    }

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    _, val_accuracies = train_and_evaluate(config, train_loader, val_loader, device)
    return max(val_accuracies)

# Run Optuna study
start_study_time = time.time()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

end_study_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.6f} seconds")

# Retrieve best results
best_trial = study.best_trial
print("Best trial:")
print(f"  Value: {best_trial.value}")
print("  Params:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

# Visualization
trials = study.trials
val_accuracies = [trial.value for trial in trials]
configs = [trial.params for trial in trials]

# Plot results
plt.figure(figsize=(12, 6))
plt.bar(range(len(configs)), val_accuracies, tick_label=[f"Trial {i + 1}" for i in range(len(configs))])
plt.title("Validation Accuracies Across Trials")
plt.xlabel("Trials")
plt.ylabel("Accuracy")
plt.show()

[I 2024-12-28 18:27:07,763] A new study created in memory with name: no-name-fbdc4e6e-18b1-4ebd-921c-05e4a7e38521
[W 2024-12-28 18:27:31,366] Trial 0 failed with parameters: {'conv_layers': [16, 32, 64], 'kernel_size': 5, 'learning_rate': 0.0007850457369697414} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/quentin/BigData/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_913/1799750954.py", line 130, in objective
    _, val_accuracies = train_and_evaluate(config, train_loader, val_loader, device)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_913/1799750954.py", line 91, in train_and_evaluate
    for inputs, labels in train_loader:
  File "/home/quentin/BigData/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 701, in __ne

KeyboardInterrupt: 

## Run the configurations